# Calculation of Self Solvation Free Energy

## Do Imports

In [1]:
import os
from rdkit import Chem
import pygromos
from pygromos.files.forcefield.openff.openff import OpenFF
from pygromos.files.gromos_system.gromos_system import Gromos_System
from pygromos.simulations.approaches.solvation_free_energy_calculation.solvation_free_energy import Solvation_free_energy_calculation
from pygromos.simulations.hpc_queuing.submission_systems.local import LOCAL

## Choose Molecule to run calculation for

In [2]:
smiles = "c1ccccc1"
workfolder = project_dir = os.path.abspath(os.path.dirname(pygromos.__file__)+"/../examples/example_files/solvation_free_energy")

### create the gromos_system from a smile and get the number of atoms

In [3]:
groSys = Gromos_System(work_folder=workfolder, system_name="test", in_smiles=smiles, forcefield=OpenFF(), auto_convert=True)
number_of_atoms = groSys.mol.GetNumAtoms()
print("Number of atoms:", number_of_atoms)

Found off: /home/runner/work/PyGromosTools/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


Found off: /home/runner/work/PyGromosTools/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


Number of atoms: 12


/bin/bash: /home/runner/micromamba/envs/pygromosTest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/runner/micromamba/envs/pygromosTest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/runner/micromamba/envs/pygromosTest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/runner/micromamba/envs/pygromosTest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/runner/micromamba/envs/pygromosTest/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/runner/micromamba/envs/pygromosTest/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [4]:
subSys = LOCAL() # use the local submission system (for cluster use LSF)

In [5]:
n_points = 5 # Number of Lambda points to calculate (typically 21)

## Create The Solvation free energy calculation system

In [6]:
sf = Solvation_free_energy_calculation(input_system=groSys, # Gromos_System, SMILES (str) or rdkit Mol
                                       work_folder=workfolder, # Folder to do calculations in
                                       system_name="test", # Name of the system (does not need to be smiles but convenient)
                                       forcefield=OpenFF(), # Force field to use
                                       density=789, # density of the liquid in kg/L
                                       num_molecules=512, # number of molecules used for the calculation
                                       num_atoms=number_of_atoms, # number of atoms in one molecule
                                       subsystem=subSys, # Subsystem to use for calculation local or lsf
                                       amberscaling=False, # Whether to use amberscaling (for openforcefield recommended)
                                       n_points=n_points) # Number of Lambda points to calculate (typically 21)

Found off: /home/runner/work/PyGromosTools/PyGromosTools/pygromos/data/ff/SMIRNOFF/openff_unconstrained-2.0.0.offxml


File imd is empty , can not be written!


/home/runner/work/PyGromosTools/PyGromosTools/pygromos/simulations/approaches/solvation_free_energy_calculation/solvation_free_energy.py:146: UserWarning: Folder does already exist
  warnings.warn("Folder does already exist")
/home/runner/work/PyGromosTools/PyGromosTools/pygromos/files/gromos_system/gromos_system.py:869: UserWarning: Did not change file path as its only promised None
  warnings.warn("Did not change file path as its only promised " + str(file_obj.path))


### Create Liquid

In [7]:
sf.create_liq()

### Minimize Liquid

In [8]:
emin_sys, jobID = sf.minimize_liq(in_gromos_simulation_system=sf.groSys_liq,prev_JobID=-1)

################################################################################

test_emin
################################################################################

Script:  /home/runner/work/PyGromosTools/PyGromosTools/pygromos/simulations/hpc_queuing/job_scheduling/schedulers/simulation_scheduler.py

################################################################################
Simulation Setup:
################################################################################

steps_per_run:  20000
equis:  0
simulation runs:  1


#####################################################################################
		ERROR in Submission
#####################################################################################


Traceback (most recent call last):
  File "/home/runner/work/PyGromosTools/PyGromosTools/pygromos/simulations/hpc_queuing/submission_systems/local.py", line 71, in submit_to_queue
    process = bash.execute(command=command, catch_STD=True, env=self.environment)
  File "/home/runner/work/PyGromosTools/PyGromosTools/pygromos/utils/bash.py", line 935, in execute
    return execute_subprocess(command=command, verbose=verbose, catch_STD=catch_STD, env=env)
  File "/home/runner/work/PyGromosTools/PyGromosTools/pygromos/utils/bash.py", line 829, in execute_subprocess
    raise ChildProcessError(msg)
ChildProcessError: SubProcess Failed due to returncode: 1
 COMMAND: 
	/home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/solvation_free_energy/test_liq/test_emin/simulation/test_emin_1/job_test_emin_1.sh
STDOUT:
	b'Could not find TMPDIR!\n'
	b' Switched to outdir for work\n'
	b'workDIR: /home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/solvation_free_energy/tes

Exception: Could not submit the commands
	ERROR in SUBMISSION!

### Change the number of cores for longer runs

In [9]:
sf.subsystem.nomp = 6

## Equilibrate System

In [10]:
eq_sys, jobID = sf.eq_liq(in_gromos_simulation_system=emin_sys,prev_JobID=jobID)

NameError: name 'emin_sys' is not defined

## Do TI calculation

In [11]:
ti_sys, jobID = sf.ti_liq(in_gromos_simulation_system=eq_sys,
                          prev_JobID=jobID,
                          n_points=n_points)

NameError: name 'eq_sys' is not defined

## Read out results

In [12]:
sf.calculate_solvation_free_energy(n_points=n_points)

/home/runner/work/PyGromosTools/PyGromosTools/examples/example_files/solvation_free_energy/test_liq/ti/test_L0.0_2/test_L0.0_2/simulation/test_L0.0_2_1/test_L0.0_2


NameError: name 'exit' is not defined